In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import platform

In [2]:
# use csv for ease of maintenance
df_ourTravel = pd.read_csv('./our travel.csv', encoding='utf8')
world = gpd.read_file('./maps/world-countries.json')

Read travel data from csv and plot.

In [3]:
listTripID = df_ourTravel.TripID.unique()
listPath = []
listCountry = []
listMarker = []
listCircle = []
listState =[]
listDoNoRepeat = []

# trip is grouped by TripID
# IMPORTANT! TripID cannot contain any space, use underscore instead
for trip in listTripID:
    
    # get all rows of each group and fetch the coordinates of each point
    TripIDGrp = df_ourTravel.loc[df_ourTravel.TripID == trip]
    Coorstr = TripIDGrp.loc[:,'Coordinates'].values
    
    # Create point coordinates for path
    Coordinates = []
    for cc in Coorstr: # make polyline
        location  = [0.0, 0.0]
        coord = cc.removeprefix('[').removesuffix(']').split(',')
        location[0] = float(coord[0])
        location[1] = float(coord[1])
        Coordinates.append(location)

    # Create paths
    fg = folium.vector_layers.PolyLine(Coordinates, popup='<b>{}</b>'.format(trip),\
        tooltip=trip, color=TripIDGrp.RouteColor.unique()[0], weight=3, opacity=0.3)
    listPath.append(fg)

    # Create countries to highlight
    countries = TripIDGrp.Country.unique()
    countries = [x for x in countries if str(x)!='nan']
    for cc in countries:
        if not(cc in listDoNoRepeat):
            geodf = world.loc[world.ADMIN == cc]
            cho = folium.Choropleth(geodf, ADMIN=cc, \
                fill_color= np.unique(TripIDGrp.loc[TripIDGrp.Country == cc].CountryColor)[0],\
                key_on='ADMIN', fill_opacity=0.1, line_opacity=0.3) 
            listCountry.append(cho)
            listDoNoRepeat.append(cc)

    for i, r in TripIDGrp.iterrows():
        
        # Create marker highlight
        if r.Marker:
            location  = [0.0, 0.0]
            coord = r.Coordinates
            coord = coord.removeprefix('[').removesuffix(']').split(',')
            location[0] = float(coord[0])
            location[1] = float(coord[1])
            r.Coordinates = location
            listMarker.append(r)
        
        # Create circle highlight
        if r.Circle:
            location  = [0.0, 0.0]
            coord = r.Coordinates
            coord = coord.removeprefix('[').removesuffix(']').split(',')
            location[0] = float(coord[0])
            location[1] = float(coord[1])
            r.Coordinates = location
            listCircle.append(r)


In [4]:
# Create map layer
ourTravel = folium.Map(location=[37.59420546987477, 61.85427147722974], tiles='openstreetmap', zoom_start= 3)
folium.TileLayer('https://{s}.basemaps.cartocdn.com/rastertiles/voyager/{z}/{x}/{y}.png', \
   attr='CartoDB.Voyager', name='CartoDB(English)').add_to(ourTravel)
folium.TileLayer('https://stamen-tiles-{s}.a.ssl.fastly.net/terrain/{z}/{x}/{y}.png', \
	attr='Stamen Terrain', name='Terrian features').add_to(ourTravel)

# Add country layer
fgCountry = folium.FeatureGroup('Country')
for ii in range(0, len(listCountry)):
    listCountry[ii].add_to(fgCountry)
fgCountry.add_to(ourTravel)

# Add marker layer
fgMarker = folium.FeatureGroup('Marker')
for ii in range(0, len(listMarker)):
    popup = folium.Popup(listMarker[ii].Popup, min_width=60, max_width=200)
    folium.Marker(location=listMarker[ii].Coordinates ,popup=popup,\
        tooltip=listMarker[ii].Tooltip,\
            icon=folium.Icon(color=listMarker[ii].MarkerColor,icon='plane')).add_to(fgMarker)
fgMarker.add_to(ourTravel)

# Add circular marker layer
fgCircle = folium.FeatureGroup('Circle')
for ii in range(0, len(listCircle)):
    popup = folium.Popup(listCircle[ii].Popup, min_width=60, max_width=200)
    folium.Circle(location=listCircle[ii].Coordinates ,popup=popup,\
        tooltip=listCircle[ii].Tooltip, radius=500, color=listCircle[ii].MarkerColor).add_to(fgCircle)
fgCircle.add_to(ourTravel)

# Add path or route layer
fgPath = folium.FeatureGroup('Travel Route')
for ii, trip in enumerate(listTripID):
    listPath[ii].add_to(fgPath)
fgPath.add_to(ourTravel)

# Add layer control legend
folium.LayerControl().add_to(ourTravel)
# Save resulting map as html
ourTravel.save('ourTravelmap.html')
# Show map
ourTravel